In [1]:
#Endurance - identify harmonics of the Bernstein waves

import sys 
sys.path.append('/Users/abrenema/Desktop/code/Aaron/github/mission_routines/rockets/Endurance/')
import end_load_data
import end_functions as end
sys.path.append('/Users/abrenema/Desktop/code/Aaron/github/plasma-physics-general/')
import plasma_params_get_density_from_flhr_freq as dflh
import plasma_params_get_flhr_freq as dflh2
sys.path.append('/Users/abrenema/Desktop/code/Aaron/github/signal_analysis/')
import plot_spectrogram as ps
from scipy import signal
from scipy.interpolate import interp1d
import numpy as np 
import matplotlib.pyplot as plt
import plasmapy
from astropy import units as u  

#%load_ext nb_black
plt.rcParams['figure.figsize'] = [10, 4]

"""Enable auto module reloading"""
#%load_ext autoreload
#%autoreload 2


"""Load E-field VLF data"""
evlf = end_load_data.efield_vlf()
print(evlf.keys())

"""Load mag data"""
mag = end_load_data.mag_dc()




#"""Load E-field HF data"""
#ehf = end_load_data.efield_hf()


%matplotlib inline
%matplotlib qt   
#Get rid of inline plots (don't work well for subplots)




dict_keys(['tvlf', 'dvlf12_mvm', 'dvlf34_mvm', 'dvlf24_mvm', 'dvlf32_mvm', 'author', 'calnote_vlf1234', 'calnote_vlf2432', 'dataunits', 'flight', 'format', 'filein', 'link', 'samplerate', 't0', 'timetagmethod', 'timeunits', 'dvlf12_mvm_gpcal', 'dvlf34_mvm_gpcal', 'dvlf32_mvm_gpcal', 'dvlf24_mvm_gpcal'])


In [2]:
#Plot VLF spectra with overplotted ion cyclotron harmonic lines 

fs = evlf.samplerate
freq12, tspec12, power12 = signal.spectrogram(evlf.dvlf12_mvm_gpcal, fs, nperseg=16384, noverlap=16384/2., return_onesided=True)

"""
#Left cyclotron waves - zoomed out
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-60], xr=[100,250],yr=[500,12000],zscale='log')
#Left cyclotron waves - zoomed in
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-50], xr=[100,250],yr=[4500,8000],zscale='log')


#Right cyclotron waves - zoomed out
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-60], xr=[600,900],yr=[50,12000],zscale='log')
#Right cyclotron waves - zoomed in
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-50], xr=[700,900],yr=[4000,9000],zscale='log')
#Right cyclotron waves - zoomed in to end (log yscale)
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-50], xr=[850,900],yr=[30,12000],zscale='log',yscale='log')
#Right cyclotron waves - zoomed in to end (linear yscale)
fig,ax = plt.subplots(2)
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-60], xr=[860,895],yr=[30,8000],zscale='log',ax=ax[0])
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-60], xr=[860,895],yr=[30,1000],zscale='log',ax=ax[1])
"""

#central cyclotron waves - zoomed out
fig,ax = plt.subplots(3)
ax[1].set_ylim(100,12000)
ax[1].set_yscale('log')
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-65],xr=[100,900],yr=[2000,9000],zscale='log',ax=ax[0])
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-65],xr=[100,900],zscale='log',ax=ax[1])
ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-65],xr=[100,900],yr=[30,1000],zscale='log',ax=ax[2])



#ps.plot_spectrogram(tspec12,freq12,power12,vr=[-75,-65],xr=[100,900],yr=[100,12000],zscale='log',yscale='log')

#Plot low freqs to find non-contaminated times 
#ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-40], xr=[100,900],yr=[1,600],zscale='log',yscale='log')
#ps.plot_spectrogram(tspec12,freq12,power12,vr=vr[-80,-40], xr=[100,900],yr=[5000,9000],zscale='log',yscale='linear')
#ps.plot_spectrogram(tspec12,freq12,power12,vr=vr[-80,-40], xr=[100,900],yr=[5000,9000],zscale='log',yscale='linear')



/Users/abrenema/Desktop/code/Aaron/github/signal_analysis/plot_spectrogram.py:94: UserWarning: Attempt to set non-positive ylim on a log-scaled axis will be ignored.
  ax.set_ylim(yr[0],yr[1])


array([[-15.34805983, -20.81218995,  -5.94525215, ..., -30.824362  ,
        -31.95074829, -36.90689196],
       [-21.53411472, -19.58618203, -22.16549364, ..., -53.11704176,
        -37.14383263, -69.59042435],
       [-14.58101643, -11.27027822, -13.66158527, ..., -32.03068356,
        -35.07363864, -37.7778742 ],
       ...,
       [-30.41017937, -34.17558049, -35.85861844, ..., -55.63627043,
        -58.06547829, -57.54183401],
       [-33.28431089, -29.30807443, -34.78148976, ..., -54.19807545,
        -57.68579979, -58.30208261],
       [-38.3032244 , -45.13586644, -31.88026186, ..., -40.94033462,
        -42.9250001 , -42.63060806]])

In [3]:
#Get cyclotron freqs

Bo = signal.decimate(mag.Bmag, 10)
tvals = signal.decimate(mag.tsec,10)
fce = [28*i for i in Bo]
fcH = [i/1836.15 for i in fce]
fcO = [i/(1836.15*15) for i in fce]
fcN = [i/(1836.15*14) for i in fce]
fcHe = [i/(1836.15*4) for i in fce]


In [4]:
#Load ion composition data from end_determine_composition.py

path = '/Users/abrenema/Desktop/Research/Rocket_missions/Endurance/data/plasma_composition/plasma_composition.pkl'
ion_comp = pickle.load(open(path, 'rb'))



plt.plot(ion_comp['times'],ion_comp['ne_langmuirprobe'],'.',ion_comp['times'],ion_comp['ne_flhID_HpOp'],'x')
plt.show()
print("here")


#----------------------------------------------
#Rough altitude comparison with IRI model O+ % 


import pandas as pd 

path = '/Users/abrenema/Desktop/Research/Rocket_missions/Endurance/data/'
folder = 'ephemeris'
fn = 'Endurance_GPS_velocity_position_altitude.csv'

header = ['time','xvel','yvel','zvel','lat','long','alt']
ephem =  pd.read_csv(path + folder + '/' + fn, skiprows=1, names=header)


#IRI O+ % 
alt_iri = [100.00,150.00,200.00,250.00,300.00,350.00,400.00,450.00,500.00,550.00,600.00,650.00,700.00,750.00,800.00,850.00,900.00]
Op_iri = [0.0,1.2,24.7,76.1,91.3,96.8,96.2,95.5,94.7,93.8,93.1,92.5,91.7,90.8,89.7,88.4,87.0]
Hp_iri = [0.0,0.0,0.0,0.0,0.0,0.1,0.2,0.3,0.4,0.7,1.0,1.3,1.7,2.2,2.9,3.6,4.4]
Op_iri = [i/100 for i in Op_iri]
Hp_iri = [i/100 for i in Hp_iri]

#Associate rocket times with altitudes so I can compare to IRI
altv = np.asarray(ephem.alt)
alts = np.empty(len(tlh))
for i in range(len(tlh)):
    good = np.where(ephem.time >= tlh[i])
    good2 = good[0][0]
    alts[i] = altv[good2]


plt.plot(alt_iri, Op_iri, alts, nO_ne)
plt.show()


h


In [ ]:
#Estimate density from lower hybrid line. Need to do this first b/c I'm not sure which line to select for upper hybrid. 

###RESULT: THIS IS GIVING ME VALUES THAT DON'T LINE UP TO THE UPPER HYBRID NOISE. MOST OF THE UH AND FPE FREQS ARE TOO HIGH.

#fs = evlf.samplerate
#freq12, tspec12, power12 = signal.spectrogram(evlf.dvlf12_mvm, fs, nperseg=16384, noverlap=16384/2., return_onesided=True)
#ps.plot_spectrogram(tspec12,freq12,power12,vr=[-80,-60], xr=[300,500],yr=[4000,10000],zscale='log')


#flh identification 
tlh = [117,139,167,206,264,335,420,479,574,666,759,817,850,868,879]
flh = [7538,8241,7882,7338,6956,6613,6476,6438,6632,6997,7426,7858,7515,5733,4711] #* u.Hz


print(range(len(tlh)))

fce = []
Bo = []
for i in range(len(tlh)):
    goo = np.squeeze(np.where(mag.tsec > tlh[i]))
    fce.append(28.*mag.Bmag[goo[0]])
    Bo.append(mag.Bmag[goo[0]])


flh = flh * u.Hz 
fce = fce * u.Hz 
Bo = Bo * u.nT



for i in range(len(tlh)): print(tlh[i], flh[i], flh[i]*1.3)



nH_ne = np.full(len(tlh),0.02) * u.dimensionless_unscaled 
nO_ne = np.full(len(tlh),0.98) * u.dimensionless_unscaled 


ne1 = dflh.dens_IonMassFractions(flh, fce, nH_ne, nO_ne)
ne2 = dflh.dens_singleion(flh, Bo, 'H+')
ne3 = dflh.dens_singleion(flh, Bo, 'O+')


ne1 = [i.value if not np.isnan(i) else 0 for i in ne1]
ne2 = [i.value if not np.isnan(i) else 0 for i in ne2]
ne3 = [i.value if not np.isnan(i) else 0 for i in ne3]

print(np.max(ne1)/1e5, np.max(ne2)/1e5, np.max(ne3)/1e5)






fpe1 = [8980*np.sqrt(i) for i in ne1]
fpe2 = [8980*np.sqrt(i) for i in ne2]
fpe3 = [8980*np.sqrt(i) for i in ne3]

fce = [i.value for i in fce]


fuh1 = [np.sqrt(fpe1[i]**2 + fce[i]**2) for i in range(len(fce))]
fuh2 = [np.sqrt(fpe2[i]**2 + fce[i]**2) for i in range(len(fce))]
fuh3 = [np.sqrt(fpe3[i]**2 + fce[i]**2) for i in range(len(fce))]

#fL=0 cutoff 
fL01 = [-1*(fce[i]/2) + np.sqrt(fpe1[i]**2 + (fce[i]/2.)**2) for i in range(len(fce))]
fL02 = [-1*(fce[i]/2) + np.sqrt(fpe2[i]**2 + (fce[i]/2.)**2) for i in range(len(fce))]
fL03 = [-1*(fce[i]/2) + np.sqrt(fpe3[i]**2 + (fce[i]/2.)**2) for i in range(len(fce))]


for i in range(len(fce)):
    print(tlh[i], fL03[i]/1e6, fce[i]/1e6, fpe3[i]/1e6, fuh3[i]/1e6)




vr = [-28,-25]
yr = [2e3, 2.6e6]
xr = [100,900]

fig, axs = plt.subplots()

axs.plot(tvals, fcet)
axs.set_ylim(yr[0],yr[1])
axs.set_xlim(xr[0],xr[1])

axs.plot(tlh, fuh3)
axs.plot(tlh, fL03)


ps.plot_spectrogram(hftimes, hffreqs, hf12,vr=vr, yscale='linear', zscale='linear', xr=xr, yr=yr, ax=axs)



